In [1]:
%%bash

./env.sh

  Using cached Pillow-9.3.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 6.2.1
    Uninstalling Pillow-6.2.1:
      Successfully uninstalled Pillow-6.2.1
  Using cached Pillow-6.2.1-cp38-cp38-manylinux1_x86_64.whl (2.1 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import json
import os
import random
import time
import pdb

import torch
import torch.backends.cudnn as cudnn

In [3]:
import os
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist

from lib.models.backbones.backbone_selector import BackboneSelector
from lib.models.tools.module_helper import ModuleHelper
from lib.models.modules.projection import ProjectionHead
from lib.utils.tools.logger import Logger as Log
from lib.models.modules.hanet_attention import HANet_Conv
from lib.models.modules.contrast import momentum_update, l2_normalize, ProjectionHead
from lib.models.modules.sinkhorn import distributed_sinkhorn
from timm.models.layers import trunc_normal_
from einops import rearrange, repeat

In [4]:
class CFG:
    model_method = 'fcn'
    phase = 'infer'

In [5]:
from lib.models.nets.hrnet import HRNet_W48_Proto
import argparse

DATA_ROOT = '/tmp/working/workspace/ProtoSeg_local/data'
SCRATCH_ROOT = '/tmp/working/workspace/ProtoSeg_local/output'
ASSET_ROOT = '/tmp/working/workspace/ProtoSeg_local/checkpoints/cityscapes/'

BACKBONE="hrnet48"
DATA_DIR = DATA_ROOT + '/Cityscapes'
# SAVE_DIR = SCRATCH_ROOT + '/Cityscapes/seg_results/'
# SAVE_DIR = '/tmp/working/workspace/PP_seg/output/Cityscapes/seg_results/'
SAVE_DIR = '/tmp/working/workspace/ProtoSeg_local/output/Cityscapes/seg_results/'

CHECKPOINTS_ROOT = SCRATCH_ROOT + "/Cityscapes"

MODEL_NAME="hrnet_w48_proto"
LOSS_TYPE="pixel_prototype_ce_loss"
CHECKPOINTS_NAME = "hrnet_w48_proto_lr1x_hrnet_proto_80k"

def main(): 
    def str2bool(v):
        """ Usage:
        parser.add_argument('--pretrained', type=str2bool, nargs='?', const=True,
                            dest='pretrained', help='Whether to use pretrained models.')
        """
        if v.lower() in ('yes', 'true', 't', 'y', '1'):
            return True
        elif v.lower() in ('no', 'false', 'f', 'n', '0'):
            return False
        else:
            raise argparse.ArgumentTypeError('Unsupported value encountered.')


    # load model
    parser.add_argument('--phase', default='train', type=str, dest='phase')
    parser.add_argument('--model_method', default='fcn', type=str, dest='model_method')
    parser.add_argument('--network', default='hrnet_w48_proto', type=str, dest='network')
    parser.add_argument('--model_name', default="hrnet_w48_proto", type=str, dest='model_name')
    parser.add_argument('--configs', default='configs/cityscapes/H_48_D_4_proto.json', type=str,dest='configs', help='The file of the hyper parameters.')
    parser.add_argument('REMAIN', nargs='*')
    parser.add_argument('--gpu', default=[0, 1, 2, 3], nargs='+', type=int, dest='gpu', help='The gpu list used.')
    parser.add_argument('--test_img', default=None, type=str, dest='test:test_img', help='The test path of image.')
    parser.add_argument('--test_dir', default=None, type=str, dest='test:test_dir', help='The test directory of images.')
    parser.add_argument('--out_dir', default='none', type=str, dest='test:out_dir', help='The test out directory of images.')
    parser.add_argument('--save_prob', type=str2bool, nargs='?', default=False, dest='test:save_prob', help='Save the logits map during testing.')
    parser.add_argument('--drop_last', type=str2bool, nargs='?', default=False, dest='data:drop_last', help='Fix bug for syncbn.')
    parser.add_argument('--data_dir', default=None, type=str, nargs='+', dest='data:data_dir', help='The Directory of the data.')
    parser.add_argument('--backbone', default=None, type=str, dest='network:backbone', help='The base network of model.')
    parser.add_argument('--checkpoints_name', default='hrnet_w48_proto_lr1x_hrnet_proto_80k', type=str, dest='checkpoints:checkpoints_name', help='The name of checkpoint model.')
    parser.add_argument('--resume', default=None, type=str, dest='network:resume', help='The path of checkpoints.')
    parser.add_argument('--loss_type', default='pixel_prototype_ce_loss', type=str, dest='loss:loss_type', help='The loss type of the network.')

    # start inference


    # python
    #args = parser.parse_args()

    # jupyter
    # args_parser = parser.parse_args(args=['--phase',
    #                                       '--model_method',
    #                                       '--network',
    #                                       '--model_name'])

    args_parser = parser.parse_args(args=['--phase', 'test',
                                         '--test_dir', DATA_DIR + '/val/image',
                                         '--out_dir', SAVE_DIR + CHECKPOINTS_NAME + '_val_ms',
                                         '--drop_last','y',
                                         '--data_dir',DATA_DIR,
                                         '--backbone', BACKBONE,
                                         '--model_name',MODEL_NAME,
                                         '--resume',CHECKPOINTS_ROOT+'/checkpoints/cityscapes/'+CHECKPOINTS_NAME+'_latest.pth',
                                         '--loss_type ',LOSS_TYPE])
    
    return args_parser



In [6]:
def inference_fn():
    parser.add_argument('--gt_dir', default=None, type=str, dest='gt_dir', help='The directory of ground truth.')
    parser.add_argument('--pred_dir', default=None, type=str, dest='pred_dir', help='The directory of predicted labels.')

    args = parser.parse_args(args=['--pred_dir',SAVE_DIR + CHECKPOINTS_NAME + '_val_ms/label',
                                   '--gt_dir', DATA_DIR + '/val/label'])
    
    return args

# utils

In [7]:
import argparse
import json
import os
import sys

from ast import literal_eval

class Configer(object):

    def __init__(self, args_parser=None, configs=None, config_dict=None):
        if config_dict is not None:
            self.params_root = config_dict

        elif configs is not None:
            if not os.path.exists(configs):
                exit(0)

            json_stream = open(configs, 'r')
            self.params_root = json.load(json_stream)
            json_stream.close()

        elif args_parser is not None:
            self.args_dict = args_parser.__dict__
            self.params_root = None

            if not os.path.exists(args_parser.configs):
                print('Json Path:{} not exists!'.format(args_parser.configs))
                exit(1)

            json_stream = open(args_parser.configs, 'r')
            self.params_root = json.load(json_stream)
            json_stream.close()

            for key, value in self.args_dict.items():
                if not self.exists(*key.split(':')):
                    self.add(key.split(':'), value)
                elif value is not None:
                    self.update(key.split(':'), value)

            self._handle_remaining_args(args_parser.REMAIN)

        self.conditions = _ConditionHelper(self)


    def _handle_remaining_args(self, remain):

        def _parse_value(x: str):
            """
            We first try to parse `x` as python literal object.
            If failed, we regard x as string.
            """
            try:
                return literal_eval(x)
            except ValueError:
                return x

        def _set_value(key, value):
            """
            We directly operate on `params_root`.
            """
            remained_parts = key.split('.')
            consumed_parts = []

            parent_dict = self.params_root
            while len(remained_parts) > 1:
                cur_key = remained_parts.pop(0)
                consumed_parts.append(cur_key)

                if cur_key not in parent_dict:
                    parent_dict[cur_key] = dict()
                elif not isinstance(parent_dict[cur_key], dict):
                    sys.exit(1)
                
                parent_dict = parent_dict[cur_key]

            cur_key = remained_parts.pop(0)
            consumed_parts.append(cur_key)

            if cur_key.endswith('+'):
                cur_key = cur_key[:-1]
                target = parent_dict.get(cur_key)

                if not isinstance(target, list):
                    sys.exit(1)

                target.append(value)
                return

            existing_value = parent_dict.get(cur_key)
#             if existing_value is not None:
#             else:
            parent_dict[cur_key] = value

        assert len(remain) % 2 == 0, remain
        args = {}
        for i in range(len(remain) // 2):
            key, value = remain[2 * i: 2 * i + 2]
            _set_value(key, _parse_value(value))

    def clone(self):
        from copy import deepcopy
        return Configer(config_dict=deepcopy(self.params_root))

    def _get_caller(self):
        filename = os.path.basename(sys._getframe().f_back.f_back.f_code.co_filename)
        lineno = sys._getframe().f_back.f_back.f_lineno
        prefix = '{}, {}'.format(filename, lineno)
        return prefix

    def get(self, *key):
        if len(key) == 0:
            return self.params_root

        elif len(key) == 1:
            if key[0] in self.params_root:
                return self.params_root[key[0]]
            else:
                exit(1)

        elif len(key) == 2:
            if key[0] in self.params_root and key[1] in self.params_root[key[0]]:
                return self.params_root[key[0]][key[1]]
            else:
                exit(1)

        else:
            exit(1)

    def exists(self, *key):
        if len(key) == 1 and key[0] in self.params_root:
            return True

        if len(key) == 2 and (key[0] in self.params_root and key[1] in self.params_root[key[0]]):
            return True

        return False

    def add(self, key_tuple, value):
        if self.exists(*key_tuple):
            exit(1)

        if len(key_tuple) == 1:
            self.params_root[key_tuple[0]] = value

        elif len(key_tuple) == 2:
            if key_tuple[0] not in self.params_root:
                self.params_root[key_tuple[0]] = dict()

            self.params_root[key_tuple[0]][key_tuple[1]] = value

        else:
            exit(1)

    def update(self, key_tuple, value):
        if not self.exists(*key_tuple):
            exit(1)

        if len(key_tuple) == 1 and not isinstance(self.params_root[key_tuple[0]], dict):
            self.params_root[key_tuple[0]] = value

        elif len(key_tuple) == 2:
            self.params_root[key_tuple[0]][key_tuple[1]] = value

        else:
            exit(1)

    def resume(self, config_dict):
        self.params_root = config_dict

    def plus_one(self, *key):
        if not self.exists(*key):
            exit(1)

        if len(key) == 1 and not isinstance(self.params_root[key[0]], dict):
            self.params_root[key[0]] += 1

        elif len(key) == 2:
            self.params_root[key[0]][key[1]] += 1

        else:
            exit(1)

    def to_dict(self):
        return self.params_root


class _ConditionHelper:
    """Handy helper"""

    def __init__(self, configer):
        self.configer = configer

    @property
    def use_multi_dataset(self):
        root_dirs = self.configer.get('data', 'data_dir')
        return isinstance(root_dirs, (tuple, list)) and len(root_dirs) > 1

    @property
    def pred_sw_offset(self):
        return self.configer.exists('data', 'pred_sw_offset')

    @property
    def pred_dt_offset(self):
        return self.configer.exists('data', 'pred_dt_offset')

    @property
    def use_sw_offset(self):
        return self.configer.exists('data', 'use_sw_offset')

    @property
    def use_dt_offset(self):
        return self.configer.exists('data', 'use_dt_offset')

    @property
    def use_ground_truth(self):
        return self.config_equals(('use_ground_truth',), True)

    @property
    def pred_ml_dt_offset(self):
        return self.configer.exists('data', 'pred_ml_dt_offset')

    def loss_contains(self, name):
        return name in self.configer.get('loss', 'loss_type')

    def model_contains(self, name):
        return name in self.configer.get('network', 'model_name')

    def config_equals(self, key, value):
        if not self.configer.exists(*key):
            return False

        return self.configer.get(*key) == value

    def config_exists(self, key):
        return self.configer.exists(*key)

    def environ_exists(self, key):
        return os.environ.get(key) is not None

    @property
    def diverse_size(self):
        return self.configer.get('val', 'data_transformer')['size_mode'] == 'diverse_size'

class AverageMeter(object):
    """ Computes ans stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [8]:
import pdb
import torch
from torch.utils import data

import lib.datasets.tools.transforms as trans
import lib.datasets.tools.cv2_aug_transforms as cv2_aug_trans
import lib.datasets.tools.pil_aug_transforms as pil_aug_trans
from lib.datasets.loader.default_loader import DefaultLoader, CSDataTestLoader
from lib.datasets.tools.collate import collate
from lib.utils.tools.logger import Logger as Log

from lib.utils.distributed import get_world_size, get_rank, is_distributed

class DataLoader(object):

    def __init__(self, configer):
        self.configer = configer

        from lib.datasets.tools import cv2_aug_transforms
        self.aug_train_transform = cv2_aug_transforms.CV2AugCompose(self.configer, split='train')
        self.aug_val_transform = cv2_aug_transforms.CV2AugCompose(self.configer, split='val')

        self.img_transform = trans.Compose([
            trans.ToTensor(),
            trans.Normalize(div_value=self.configer.get('normalize', 'div_value'),
                            mean=self.configer.get('normalize', 'mean'),
                            std=self.configer.get('normalize', 'std')), ])

        self.label_transform = trans.Compose([
            trans.ToLabel(),
            trans.ReLabel(255, -1), ])
 
    def get_dataloader_sampler(self, klass, split, dataset):

        from lib.datasets.loader.multi_dataset_loader import MultiDatasetLoader, MultiDatasetTrainingSampler

        root_dir = self.configer.get('data', 'data_dir')
        if isinstance(root_dir, list) and len(root_dir) == 1:
            root_dir = root_dir[0]

        kwargs = dict(
            dataset=dataset,
            aug_transform=(self.aug_train_transform if split == 'train' else self.aug_val_transform),
            img_transform=self.img_transform,
            label_transform=self.label_transform,
            configer=self.configer
        )

        if isinstance(root_dir, str):
            loader = klass(root_dir, **kwargs)
            multi_dataset = False
        elif isinstance(root_dir, list):
            loader = MultiDatasetLoader(root_dir, klass, **kwargs)
            multi_dataset = True
            Log.info('use multi-dataset for {}...'.format(dataset))
        else:
            raise RuntimeError('Unknown root dir {}'.format(root_dir))

        if split == 'train':
            if is_distributed() and multi_dataset:
                raise RuntimeError('Currently multi dataset doesn\'t support distributed.')

            if is_distributed():
                sampler = torch.utils.data.distributed.DistributedSampler(loader)
            elif multi_dataset:
                sampler = MultiDatasetTrainingSampler(loader)
            else:
                sampler = None

        elif split == 'val':

            if is_distributed():
                sampler = torch.utils.data.distributed.DistributedSampler(loader)
            else:
                sampler = None

        return loader, sampler

    def get_valloader(self, dataset=None):
        dataset = 'val' if dataset is None else dataset

        if self.configer.exists('data', 'use_dt_offset') or self.configer.exists('data', 'pred_dt_offset'):
            """
            dt-offset manner:
            load both the ground-truth label and offset (based on distance transform).
            """
            Log.info('use distance transform based offset loader for val ...')
            klass = DTOffsetLoader

        elif self.configer.get('method') == 'fcn_segmentor':
            """
            default manner:
            load the ground-truth label.
            """
            Log.info('use DefaultLoader for val ...')
            klass = DefaultLoader
        else:
            Log.error('Method: {} loader is invalid.'.format(self.configer.get('method')))
            return None

        loader, sampler = self.get_dataloader_sampler(klass, 'val', dataset)
        valloader = data.DataLoader(
            loader,
            sampler=sampler,
            batch_size=self.configer.get('val', 'batch_size') // get_world_size(), pin_memory=False,
            num_workers=self.configer.get('data', 'workers'), shuffle=False,
            collate_fn=lambda *args: collate(
                *args, trans_dict=self.configer.get('val', 'data_transformer')
            )
        )
        return valloader

    def get_testloader(self, dataset=None):
        dataset = 'test' if dataset is None else dataset
        if self.configer.exists('data', 'use_sw_offset') or self.configer.exists('data', 'pred_sw_offset'):
            Log.info('use sliding window based offset loader for test ...')
            test_loader = data.DataLoader(
                SWOffsetTestLoader(root_dir=self.configer.get('data', 'data_dir'), dataset=dataset,
                                   img_transform=self.img_transform,
                                   configer=self.configer),
                batch_size=self.configer.get('test', 'batch_size'), pin_memory=False,
                num_workers=self.configer.get('data', 'workers'), shuffle=False,
                collate_fn=lambda *args: collate(
                    *args, trans_dict=self.configer.get('test', 'data_transformer')
                )
            )
            return test_loader

        elif self.configer.get('method') == 'fcn_segmentor':
            Log.info('use CSDataTestLoader for test ...')

            root_dir = self.configer.get('data', 'data_dir')
            if isinstance(root_dir, list) and len(root_dir) == 1:
                root_dir = root_dir[0]
            test_loader = data.DataLoader(
                CSDataTestLoader(root_dir=root_dir, dataset=dataset,
                                 img_transform=self.img_transform,
                                 configer=self.configer),
                batch_size=self.configer.get('test', 'batch_size'), pin_memory=True,
                num_workers=self.configer.get('data', 'workers'), shuffle=False,
                collate_fn=lambda *args: collate(
                    *args, trans_dict=self.configer.get('test', 'data_transformer')
                )
            )
            return test_loader

# Model

In [9]:
class HRNet_W48_Proto(nn.Module):
    """
    deep high-resolution representation learning for human pose estimation, CVPR2019
    """

    def __init__(self, configer):
        super(HRNet_W48_Proto, self).__init__()
        self.configer = configer
        self.gamma = self.configer.get('protoseg', 'gamma')
        self.num_prototype = self.configer.get('protoseg', 'num_prototype')
        self.use_prototype = self.configer.get('protoseg', 'use_prototype')
        self.update_prototype = self.configer.get('protoseg', 'update_prototype')
        self.pretrain_prototype = self.configer.get('protoseg', 'pretrain_prototype')
        self.num_classes = self.configer.get('data', 'num_classes')
        self.backbone = BackboneSelector(configer).get_backbone()

        in_channels = 720
        self.cls_head = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
            ModuleHelper.BNReLU(in_channels, bn_type=self.configer.get('network', 'bn_type')),
            nn.Dropout2d(0.10)
        )

        self.prototypes = nn.Parameter(torch.zeros(self.num_classes, self.num_prototype, in_channels),
                                       requires_grad=True)

        self.proj_head = ProjectionHead(in_channels, in_channels)
        self.feat_norm = nn.LayerNorm(in_channels)
        self.mask_norm = nn.LayerNorm(self.num_classes)

        trunc_normal_(self.prototypes, std=0.02)

    def prototype_learning(self, _c, out_seg, gt_seg, masks):
        pred_seg = torch.max(out_seg, 1)[1]
        mask = (gt_seg == pred_seg.view(-1))

        cosine_similarity = torch.mm(_c, self.prototypes.view(-1, self.prototypes.shape[-1]).t())

        proto_logits = cosine_similarity
        proto_target = gt_seg.clone().float()

        # clustering for each class
        protos = self.prototypes.data.clone()
        for k in range(self.num_classes):
            init_q = masks[..., k]
            init_q = init_q[gt_seg == k, ...]
            if init_q.shape[0] == 0:
                continue

            q, indexs = distributed_sinkhorn(init_q)

            m_k = mask[gt_seg == k]

            c_k = _c[gt_seg == k, ...]

            m_k_tile = repeat(m_k, 'n -> n tile', tile=self.num_prototype)

            m_q = q * m_k_tile  # n x self.num_prototype

            c_k_tile = repeat(m_k, 'n -> n tile', tile=c_k.shape[-1])

            c_q = c_k * c_k_tile  # n x embedding_dim

            f = m_q.transpose(0, 1) @ c_q  # self.num_prototype x embedding_dim

            n = torch.sum(m_q, dim=0)

            if torch.sum(n) > 0 and self.update_prototype is True:
                f = F.normalize(f, p=2, dim=-1)

                new_value = momentum_update(old_value=protos[k, n != 0, :], new_value=f[n != 0, :],
                                            momentum=self.gamma, debug=False)
                protos[k, n != 0, :] = new_value

            proto_target[gt_seg == k] = indexs.float() + (self.num_prototype * k)

        self.prototypes = nn.Parameter(l2_normalize(protos),
                                       requires_grad=False)

        if dist.is_available() and dist.is_initialized():
            protos = self.prototypes.data.clone()
            dist.all_reduce(protos.div_(dist.get_world_size()))
            self.prototypes = nn.Parameter(protos, requires_grad=False)

        return proto_logits, proto_target

    def forward(self, x_, gt_semantic_seg=None, pretrain_prototype=False):
        x = self.backbone(x_)
        _, _, h, w = x[0].size()

        feat1 = x[0]
        feat2 = F.interpolate(x[1], size=(h, w), mode="bilinear", align_corners=True)
        feat3 = F.interpolate(x[2], size=(h, w), mode="bilinear", align_corners=True)
        feat4 = F.interpolate(x[3], size=(h, w), mode="bilinear", align_corners=True)

        feats = torch.cat([feat1, feat2, feat3, feat4], 1)
        c = self.cls_head(feats)

        c = self.proj_head(c)
        _c = rearrange(c, 'b c h w -> (b h w) c')
        _c = self.feat_norm(_c)
        _c = l2_normalize(_c)

        self.prototypes.data.copy_(l2_normalize(self.prototypes))

        # n: h*w, k: num_class, m: num_prototype
        masks = torch.einsum('nd,kmd->nmk', _c, self.prototypes)
        prototype = self.prototypes

        out_seg = torch.amax(masks, dim=1)
        out_seg = self.mask_norm(out_seg)
        out_seg = rearrange(out_seg, "(b h w) k -> b k h w", b=feats.shape[0], h=feats.shape[2])

        if pretrain_prototype is False and self.use_prototype is True and gt_semantic_seg is not None:
            print('pretrain_prototype & if in')
            gt_seg = F.interpolate(gt_semantic_seg.float(), size=feats.size()[2:], mode='nearest').view(-1)
            contrast_logits, contrast_target = self.prototype_learning(_c, out_seg, gt_seg, masks)
            return {'seg': out_seg, 'logits': contrast_logits, 'target': contrast_target}

        return out_seg, prototype

In [10]:
#from lib.models.nets.hrnet import HRNet_W48_Proto

SEG_MODEL_DICT = {
    'hrnet_w48_proto': HRNet_W48_Proto
}

class Segmentation_Model(object):
    def __init__(self, configer):
        self.configer = configer

    def semantic_segmentor(self):
        # ここよくわからん
        #model_name = self.configer.get('network', 'model_name')
        model_name = 'hrnet_w48_proto'

        if model_name not in SEG_MODEL_DICT:
            print("your select model not found")
        print(model_name)
#         model= SEG_MODEL_DICT[model_name](self.configer)
        model= HRNet_W48_Proto(self.configer)
        print('ok')

        return model


# Loss

In [11]:
from lib.loss.loss_helper import FSAuxOhemCELoss, FSOhemCELoss, FSRMILoss
from lib.loss.loss_helper import FSCELoss, FSAuxCELoss, FSAuxRMILoss, FSCELOVASZLoss, MSFSAuxRMILoss, FSAuxCELossDSN
from lib.loss.loss_helper import SegFixLoss
from lib.loss.rmi_loss import RMILoss
from lib.loss.loss_contrast import ContrastAuxCELoss, ContrastCELoss
from lib.loss.loss_contrast_mem import ContrastCELoss as MemContrastCELoss
from lib.loss.loss_proto import PixelPrototypeCELoss

SEG_LOSS_DICT = {
    'fs_ce_loss': FSCELoss,
    'fs_ohemce_loss': FSOhemCELoss,
    'fs_auxce_loss': FSAuxCELoss,
    'fs_aux_rmi_loss': FSAuxRMILoss,
    'fs_auxohemce_loss': FSAuxOhemCELoss,
    'segfix_loss': SegFixLoss,
    'rmi_loss': RMILoss,
    'fs_rmi_loss': FSRMILoss,
    'contrast_auxce_loss': ContrastAuxCELoss,
    'contrast_ce_loss': ContrastCELoss,
    'fs_ce_lovasz_loss': FSCELOVASZLoss,
    'ms_fs_aux_rmi_loss': MSFSAuxRMILoss,
    'fs_auxce_dsn_loss': FSAuxCELossDSN,
    'mem_contrast_ce_loss': MemContrastCELoss,
    'pixel_prototype_ce_loss': PixelPrototypeCELoss
}

class LossManager(object):
    def __init__(self, configer):
        self.configer = configer

    def _parallel(self, loss):
        if is_distributed():
            Log.info('use distributed loss')
            return loss
            
        if self.configer.get('network', 'loss_balance') and len(self.configer.get('gpu')) > 1:
            Log.info('use DataParallelCriterion loss')
            from lib.extensions.parallel.data_parallel import DataParallelCriterion
            loss = DataParallelCriterion(loss)

        return loss

    def get_seg_loss(self, loss_type=None):
        key = self.configer.get('loss', 'loss_type') if loss_type is None else loss_type
        if key not in SEG_LOSS_DICT:
            Log.error('Loss: {} not valid!'.format(key))
            exit(1)
        Log.info('use loss: {}.'.format(key))
        loss = SEG_LOSS_DICT[key](self.configer)
        return self._parallel(loss)


# optim & scheduler

In [12]:
import math

import torchcontrib
from torch.optim import SGD, Adam, AdamW, lr_scheduler
from torch.optim.lr_scheduler import LambdaLR

from lib.utils.tools.logger import Logger as Log


class WarmupCosineSchedule(LambdaLR):
    """ Linear warmup and then cosine decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps following a cosine curve.
        If `cycles` (default=0.5) is different from default, learning rate follows cosine function after warmup.
    """

    def __init__(self, optimizer, warmup_steps, t_total, cycles=.5, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        self.cycles = cycles
        super(WarmupCosineSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1.0, self.warmup_steps))
        # progress after warmup
        progress = float(step - self.warmup_steps) / float(max(1, self.t_total - self.warmup_steps))
        return max(0.0, 0.5 * (1. + math.cos(math.pi * float(self.cycles) * 2.0 * progress)))


class OptimScheduler(object):
    def __init__(self, configer):
        self.configer = configer

    def init_optimizer(self, net_params):
        optimizer = None
        if self.configer.get('optim', 'optim_method') == 'sgd':
            optimizer = SGD(net_params,
                            lr=self.configer.get('lr', 'base_lr'),
                            momentum=self.configer.get('optim', 'sgd')['momentum'],
                            weight_decay=self.configer.get('optim', 'sgd')['weight_decay'],
                            nesterov=self.configer.get('optim', 'sgd')['nesterov'])

        elif self.configer.get('optim', 'optim_method') == 'adam':
            optimizer = Adam(net_params,
                             lr=self.configer.get('lr', 'base_lr'),
                             betas=self.configer.get('optim', 'adam')['betas'],
                             eps=self.configer.get('optim', 'adam')['eps'],
                             weight_decay=self.configer.get('optim', 'adam')['weight_decay'])
        elif self.configer.get('optim', 'optim_method') == 'adamw':
            optimizer = AdamW(net_params,
                              lr=self.configer.get('lr', 'base_lr'),
                              betas=self.configer.get('optim', 'adamw')['betas'],
                              eps=self.configer.get('optim', 'adamw')['eps'],
                              weight_decay=self.configer.get('optim', 'adamw')['weight_decay'])

        else:
            Log.error('Optimizer {} is not valid.'.format(self.configer.get('optim', 'optim_method')))
            exit(1)

        policy = self.configer.get('lr', 'lr_policy')

        scheduler = None
        if policy == 'step':
            scheduler = lr_scheduler.StepLR(optimizer,
                                            self.configer.get('lr', 'step')['step_size'],
                                            gamma=self.configer.get('lr', 'step')['gamma'])

        elif policy == 'multistep':
            scheduler = lr_scheduler.MultiStepLR(optimizer,
                                                 self.configer.get('lr', 'multistep')['stepvalue'],
                                                 gamma=self.configer.get('lr', 'multistep')['gamma'])

        elif policy == 'lambda_poly':
            if os.environ.get('lambda_poly_power'):
                _lambda_poly_power = float(os.environ.get('lambda_poly_power'))
                Log.info('Use lambda_poly policy with power {}'.format(_lambda_poly_power))
                lambda_poly = lambda iters: pow((1.0 - iters / self.configer.get('solver', 'max_iters')),
                                                _lambda_poly_power)
            elif self.configer.exists('lr', 'lambda_poly'):
                Log.info('Use lambda_poly policy with power {}'.format(self.configer.get('lr', 'lambda_poly')['power']))
                lambda_poly = lambda iters: pow((1.0 - iters / self.configer.get('solver', 'max_iters')),
                                                self.configer.get('lr', 'lambda_poly')['power'])
            else:
                Log.info('Use lambda_poly policy with default power 0.9')
                lambda_poly = lambda iters: pow((1.0 - iters / self.configer.get('solver', 'max_iters')), 0.9)
            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_poly)

        elif policy == 'lambda_cosine':
            lambda_cosine = lambda iters: (math.cos(math.pi * iters / self.configer.get('solver', 'max_iters'))
                                           + 1.0) / 2
            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_cosine)

        elif policy == 'plateau':
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode=self.configer.get('lr', 'plateau')['mode'],
                                                       factor=self.configer.get('lr', 'plateau')['factor'],
                                                       patience=self.configer.get('lr', 'plateau')['patience'],
                                                       threshold=self.configer.get('lr', 'plateau')['threshold'],
                                                       threshold_mode=self.configer.get('lr', 'plateau')['thre_mode'],
                                                       cooldown=self.configer.get('lr', 'plateau')['cooldown'],
                                                       min_lr=self.configer.get('lr', 'plateau')['min_lr'],
                                                       eps=self.configer.get('lr', 'plateau')['eps'])

        elif policy == 'swa_lambda_poly':
            optimizer = torchcontrib.optim.SWA(optimizer)
            normal_max_iters = int(self.configer.get('solver', 'max_iters') * 0.75)
            swa_step_max_iters = (self.configer.get('solver',
                                                    'max_iters') - normal_max_iters) // 5 + 1  # we use 5 ensembles here

            def swa_lambda_poly(iters):
                if iters < normal_max_iters:
                    return pow(1.0 - iters / normal_max_iters, 0.9)
                else:  # set lr to half of initial lr and start swa
                    return 0.5 * pow(1.0 - ((iters - normal_max_iters) % swa_step_max_iters) / swa_step_max_iters, 0.9)

            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=swa_lambda_poly)

        elif policy == 'swa_lambda_cosine':
            optimizer = torchcontrib.optim.SWA(optimizer)
            normal_max_iters = int(self.configer.get('solver', 'max_iters') * 0.75)
            swa_step_max_iters = (self.configer.get('solver',
                                                    'max_iters') - normal_max_iters) // 5 + 1  # we use 5 ensembles here

            def swa_lambda_cosine(iters):
                if iters < normal_max_iters:
                    return (math.cos(math.pi * iters / normal_max_iters) + 1.0) / 2
                else:  # set lr to half of initial lr and start swa
                    return 0.5 * (math.cos(
                        math.pi * ((iters - normal_max_iters) % swa_step_max_iters) / swa_step_max_iters) + 1.0) / 2

            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=swa_lambda_cosine)

        elif policy == 'warmup_cosine':
            scheduler = WarmupCosineSchedule(optimizer, warmup_steps=1000,
                                             t_total=self.configer.get('solver', 'max_iters'))

        else:
            Log.error('Policy:{} is not valid.'.format(policy))
            exit(1)

        return optimizer, scheduler

    def update_optimizer(self, net, optim_method, lr_policy):
        self.configer.update(('optim', 'optim_method'), optim_method)
        self.configer.update(('lr', 'lr_policy'), lr_policy)
        optimizer, scheduler = self.init_optimizer(net)
        return optimizer, scheduler

# Inference

In [13]:
class Tester(object):
    """
      The class for Pose Estimation. Include train, val, val & predict.
    """

    def __init__(self, configer):
        self.configer = configer
        self.batch_time = AverageMeter()
        self.data_time = AverageMeter()
        self.seg_visualizer = SegVisualizer(configer)
        self.loss_manager = LossManager(configer)
        self.module_runner = ModuleRunner(configer)
        self.model_manager = Segmentation_Model(configer)
        self.optim_scheduler = OptimScheduler(configer)
        self.seg_data_loader = DataLoader(configer)
        self.save_dir = self.configer.get('test', 'out_dir')
        self.seg_net = None
        self.test_loader = None
        self.test_size = None
        self.infer_time = 0
        self.infer_cnt = 0
        self._init_model()

    def _init_model(self):
        self.seg_net = self.model_manager.semantic_segmentor()
        self.seg_net = self.module_runner.load_net(self.seg_net)
        

        if 'test' in self.save_dir:
            self.test_loader = self.seg_data_loader.get_testloader()
            self.test_size = len(self.test_loader) * self.configer.get('test', 'batch_size')
        else:
            self.test_loader = self.seg_data_loader.get_valloader()
            self.test_size = len(self.test_loader) * self.configer.get('val', 'batch_size')

        self.seg_net.eval()

    def __relabel(self, label_map):
        height, width = label_map.shape
        label_dst = np.zeros((height, width), dtype=np.uint8)
        for i in range(self.configer.get('data', 'num_classes')):
            label_dst[label_map == i] = self.configer.get('data', 'label_list')[i]

        label_dst = np.array(label_dst, dtype=np.uint8)

        return label_dst

    def test(self, data_loader=None):
        """
          Validation function during the train phase.
        """
        self.seg_net.eval()
        start_time = time.time()
        image_id = 0


        FileHelper.make_dirs(self.save_dir, is_file=False)

        if self.configer.get('dataset') in ['cityscapes', 'gta5', 'woodscape']:
            colors = get_cityscapes_colors()
        else:
            raise RuntimeError("Unsupport colors")

        save_prob = False
        if self.configer.get('test', 'save_prob'):
            save_prob = self.configer.get('test', 'save_prob')

            def softmax(X, axis=0):
                max_prob = np.max(X, axis=axis, keepdims=True)
                X -= max_prob
                X = np.exp(X)
                sum_prob = np.sum(X, axis=axis, keepdims=True)
                X /= sum_prob
                return X

        for j, data_dict in enumerate(self.test_loader):
            inputs = data_dict['img']
            names = data_dict['name']
            metas = data_dict['meta']
            if 'subfolder' in data_dict:
                subfolder = data_dict['subfolder']

            if '/val/' in self.save_dir: #and os.environ.get('save_gt_label'):
                labels = data_dict['labelmap']

            with torch.no_grad():
                # Forward pass.
                print('Forward pass')
                if self.configer.exists('data', 'use_offset') and self.configer.get('data', 'use_offset') == 'offline':
                    offset_h_maps = data_dict['offsetmap_h']
                    offset_w_maps = data_dict['offsetmap_w']
                    outputs = self.offset_test(inputs, offset_h_maps, offset_w_maps)
                    print('offline')
                elif self.configer.get('test', 'mode') == 'ss_test':
                    outputs, prototype = self.ss_test(inputs)
                    print('ss_test')
                ######################################################################################################################333
                

                if isinstance(outputs, torch.Tensor):
                    outputs = outputs.permute(0, 2, 3, 1).cpu().numpy()
                    n = outputs.shape[0]
                else:
                    outputs = [output.permute(0, 2, 3, 1).cpu().numpy().squeeze() for output in outputs]
                    n = len(outputs)
                    #######################################################################################################################
                    # model output
                    
                #n is mask channel

                for k in range(n):
                    image_id += 1
                    ori_img_size = metas[k]['ori_img_size']
                    border_size = metas[k]['border_size']
                    logits = cv2.resize(outputs[k][:border_size[1], :border_size[0]],
                                        tuple(ori_img_size), interpolation=cv2.INTER_CUBIC)

                    # save the logits map
                    if self.configer.get('test', 'save_prob'):
                        prob_path = os.path.join(self.save_dir, "prob/", '{}.npy'.format(names[k]))
                        FileHelper.make_dirs(prob_path, is_file=True)
                        np.save(prob_path, softmax(logits, axis=-1))

                    label_img = np.asarray(np.argmax(logits, axis=-1), dtype=np.uint8)
                    if self.configer.exists('data', 'reduce_zero_label') and self.configer.get('data',
                                                                                               'reduce_zero_label'):
                        label_img = label_img + 1
                        label_img = label_img.astype(np.uint8)
                    if self.configer.exists('data', 'label_list'):
                        label_img_ = self.__relabel(label_img)
                    else:
                        label_img_ = label_img
                    label_img_ = Image.fromarray(label_img_, 'P')
                    if 'subfolder' not in data_dict or len(subfolder[k]) == 0:
                        label_path = os.path.join(self.save_dir, "label/", '{}.png'.format(names[k]))
                    else:
                        label_path = os.path.join(self.save_dir, "label/", '{}/{}.png'.format(subfolder[k], names[k]))

                    FileHelper.make_dirs(label_path, is_file=True)
                    ImageHelper.save(label_img_, label_path)

                    # colorize the label-map
                    if os.environ.get('save_gt_label'):
                        if self.configer.exists('data', 'reduce_zero_label') and self.configer.get('data','reduce_zero_label'):
                            label_img = labels[k]
                            label_img = np.asarray(label_img, dtype=np.uint8)
                        color_img_ = Image.fromarray(label_img)
                        color_img_.putpalette(colors)
                        vis_path = os.path.join(self.save_dir, "gt_vis/", '{}.png'.format(names[k]))
                        FileHelper.make_dirs(vis_path, is_file=True)
                        ImageHelper.save(color_img_, save_path=vis_path)
                    else:
                        color_img_ = Image.fromarray(label_img)
                        color_img_.putpalette(colors)
                        vis_path = os.path.join(self.save_dir, "vis/", '{}.png'.format(names[k]))
                        FileHelper.make_dirs(vis_path, is_file=True)
                        ImageHelper.save(color_img_, save_path=vis_path)
                        
            self.batch_time.update(time.time() - start_time)
            start_time = time.time()
            
            break
        
#         return prototype
        

    def ss_test(self, inputs, scale=1):
        if isinstance(inputs, torch.Tensor):
            n, c, h, w = inputs.size(0), inputs.size(1), inputs.size(2), inputs.size(3)
            scaled_inputs = F.interpolate(inputs, size=(int(h * scale), int(w * scale)), mode="bilinear",
                                          align_corners=True)
            start = timeit.default_timer()
            outputs, prototype = self.seg_net.forward(scaled_inputs)
            print("★★★★★★★★★★★★★★★★★★★★★★★★")
            print(prototype)
            print("★★★★★★★★★★★★★★★★★★★★★★★★")
            torch.cuda.synchronize()
            end = timeit.default_timer()

            if isinstance(outputs, list):
                outputs = outputs[-1]
            elif isinstance(outputs, dict):
                outputs = outputs['seg']
            elif isinstance(outputs, tuple):
                outputs = outputs[-1]
            outputs = F.interpolate(outputs, size=(h, w), mode='bilinear', align_corners=True)
            
            #b c h w
            print("■■■■■■■■■■■■■■■■■■■■■■■■■")
            print('outputs',outputs.shape)
            print('prototype',prototype.shape)
            print("■■■■■■■■■■■■■■■■■■■■■■■■■")
            return outputs,prototype
        
        elif isinstance(inputs, collections.Sequence):
            device_ids = self.configer.get('gpu')
            replicas = nn.parallel.replicate(self.seg_net.module, device_ids)
            scaled_inputs, ori_size, outputs = [], [], []
            for i, d in zip(inputs, device_ids):
                h, w = i.size(1), i.size(2)
                ori_size.append((h, w))
                i = F.interpolate(i.unsqueeze(0), size=(int(h * scale), int(w * scale)), mode="bilinear",
                                  align_corners=True)
                scaled_inputs.append(i.cuda(d, non_blocking=True))
            scaled_outputs = nn.parallel.parallel_apply(replicas[:len(scaled_inputs)], scaled_inputs)
            for i, output in enumerate(scaled_outputs):
                outputs.append(F.interpolate(output[-1], size=ori_size[i], mode='bilinear', align_corners=True))
            return outputs
        else:
            raise RuntimeError("Unsupport data type: {}".format(type(inputs)))


        return outputs

In [14]:
try:
    from itertools import izip
except ImportError:
    izip = zip

# Cityscapes imports
try:
    from lib.metrics.cityscapes.evaluation.csHelpers import *
except:
    from cityscapes.evaluation.csHelpers import *
    
CSUPPORT = True
# Check if C-Support is available for better performance
if CSUPPORT:
    try:
        import lib.metrics.cityscapes.evaluation.addToConfusionMatrix as addToConfusionMatrix
    except:
        CSUPPORT = False

In [15]:
class CArgs(object):
    def __init__(self, data_path=None, out_path=None, predict_path=None):
        # Where to look for Cityscapes, note that data path is equal to gt path
        if 'CITYSCAPES_DATASET' in os.environ:
            self.cityscapesPath = os.environ['CITYSCAPES_DATASET']
        else:
            self.cityscapesPath = os.path.join(data_path)

        if 'CITYSCAPES_EXPORT_DIR' in os.environ:
            export_dir = os.environ['CITYSCAPES_EXPORT_DIR']
            if not os.path.isdir(export_dir):
                raise ValueError("CITYSCAPES_EXPORT_DIR {} is not a directory".format(export_dir))
            self.exportFile = "{}/resultPixelLevelSemanticLabeling.json".format(export_dir)
        else:
            self.exportFile = os.path.join(out_path, "evaluationResults", "resultPixelLevelSemanticLabeling.json")
        # Parameters that should be modified by user
        self.groundTruthSearch  = os.path.join( self.cityscapesPath, "*.png" )

        # Remaining params
        self.evalInstLevelScore = True
        self.evalPixelAccuracy  = True
        self.evalLabels         = []
        self.printRow           = 5
        self.normalized         = True
        self.colorized          = hasattr(sys.stderr, "isatty") and sys.stderr.isatty() and platform.system()=='Linux'
        self.bold               = colors.BOLD if self.colorized else ""
        self.nocol              = colors.ENDC if self.colorized else ""
        self.JSONOutput         = True
        self.quiet              = False

        self.avgClassSize       = {
        "bicycle"    :  4672.3249222261 ,
        "caravan"    : 36771.8241758242 ,
        "motorcycle" :  6298.7200839748 ,
        "rider"      :  3930.4788056518 ,
        "bus"        : 35732.1511111111 ,
        "train"      : 67583.7075812274 ,
        "car"        : 12794.0202738185 ,
        "person"     :  3462.4756337644 ,
        "truck"      : 27855.1264367816 ,
        "trailer"    : 16926.9763313609 ,
    }

        # store some parameters for finding predictions in the args variable
        # the values are filled when the method getPrediction is first called
        self.predictionPath = predict_path
        self.predictionWalk = None
## method part
def getPrediction( args, groundTruthFile ):
    # determine the prediction path, if the method is first called
    if not args.predictionPath:
        rootPath = None
        if 'CITYSCAPES_RESULTS' in os.environ:
            rootPath = os.environ['CITYSCAPES_RESULTS']
        elif 'CITYSCAPES_DATASET' in os.environ:
            rootPath = os.path.join( os.environ['CITYSCAPES_DATASET'] , "results" )
        else:
            rootPath = os.path.join(os.path.dirname(os.path.realpath(__file__)),'..','..','results')

        if not os.path.isdir(rootPath):
            printError("Could not find a result root folder. Please read the instructions of this method.")

        args.predictionPath = rootPath

    # walk the prediction path, if not happened yet
    if not args.predictionWalk:
        walk = []
        for root, dirnames, filenames in os.walk(args.predictionPath):
            walk.append( (root,filenames) )
        args.predictionWalk = walk

    csFile = getCsFileInfo(groundTruthFile)
    filePattern = "{}_{}_{}*.png".format( csFile.city , csFile.sequenceNb , csFile.frameNb )

    predictionFile = None
    for root, filenames in args.predictionWalk:
        for filename in fnmatch.filter(filenames, filePattern):
            if not predictionFile:
                predictionFile = os.path.join(root, filename)
            else:
                printError("Found multiple predictions for ground truth {}".format(groundTruthFile))

    if not predictionFile:
        printError("Found no prediction for ground truth {}".format(groundTruthFile))

    return predictionFile

# Generate empty confusion matrix and create list of relevant labels
def generateMatrix(args):
    args.evalLabels = []
    for label in labels:
        if (label.id < 0):
            continue
        # we append all found labels, regardless of being ignored
        args.evalLabels.append(label.id)
    maxId = max(args.evalLabels)
    # We use longlong type to be sure that there are no overflows
    return np.zeros(shape=(maxId + 1, maxId + 1), dtype=np.ulonglong)


def generateInstanceStats(args):
    instanceStats = {}
    instanceStats["classes"] = {}
    instanceStats["categories"] = {}
    for label in labels:
        if label.hasInstances and not label.ignoreInEval:
            instanceStats["classes"][label.name] = {}
            instanceStats["classes"][label.name]["tp"] = 0.0
            instanceStats["classes"][label.name]["tpWeighted"] = 0.0
            instanceStats["classes"][label.name]["fn"] = 0.0
            instanceStats["classes"][label.name]["fnWeighted"] = 0.0
    for category in category2labels:
        labelIds = []
        allInstances = True
        for label in category2labels[category]:
            if label.id < 0:
                continue
            if not label.hasInstances:
                allInstances = False
                break
            labelIds.append(label.id)
        if not allInstances:
            continue

        instanceStats["categories"][category] = {}
        instanceStats["categories"][category]["tp"] = 0.0
        instanceStats["categories"][category]["tpWeighted"] = 0.0
        instanceStats["categories"][category]["fn"] = 0.0
        instanceStats["categories"][category]["fnWeighted"] = 0.0
        instanceStats["categories"][category]["labelIds"] = labelIds

    return instanceStats


# Get absolute or normalized value from field in confusion matrix.
def getMatrixFieldValue(confMatrix, i, j, args):
    if args.normalized:
        rowSum = confMatrix[i].sum()
        if (rowSum == 0):
            return float('nan')
        return float(confMatrix[i][j]) / rowSum
    else:
        return confMatrix[i][j]


# Calculate and return IOU score for a particular label
def getIouScoreForLabel(label, confMatrix, args):
    if id2label[label].ignoreInEval:
        return float('nan')

    # the number of true positive pixels for this label
    # the entry on the diagonal of the confusion matrix
    tp = np.longlong(confMatrix[label, label])

    # the number of false negative pixels for this label
    # the row sum of the matching row in the confusion matrix
    # minus the diagonal entry
    fn = np.longlong(confMatrix[label, :].sum()) - tp

    # the number of false positive pixels for this labels
    # Only pixels that are not on a pixel with ground truth label that is ignored
    # The column sum of the corresponding column in the confusion matrix
    # without the ignored rows and without the actual label of interest
    notIgnored = [l for l in args.evalLabels if not id2label[l].ignoreInEval and not l == label]
    fp = np.longlong(confMatrix[notIgnored, label].sum())

    # the denominator of the IOU score
    denom = (tp + fp + fn)
    if denom == 0:
        return float('nan')

    # return IOU
    return float(tp) / denom





# Calculate and return IOU score for a particular label
def getInstanceIouScoreForLabel(label, confMatrix, instStats, args):
    if id2label[label].ignoreInEval:
        return float('nan')

    labelName = id2label[label].name
    if not labelName in instStats["classes"]:
        return float('nan')

    tp = instStats["classes"][labelName]["tpWeighted"]
    fn = instStats["classes"][labelName]["fnWeighted"]
    # false postives computed as above
    notIgnored = [l for l in args.evalLabels if not id2label[l].ignoreInEval and not l == label]
    fp = np.longlong(confMatrix[notIgnored, label].sum())

    # the denominator of the IOU score
    denom = (tp + fp + fn)
    if denom == 0:
        return float('nan')

    # return IOU
    return float(tp) / denom


# Calculate prior for a particular class id.
def getPrior(label, confMatrix):
    return float(confMatrix[label, :].sum()) / confMatrix.sum()


# Get average of scores.
# Only computes the average over valid entries.
def getScoreAverage(scoreList, args):
    validScores = 0
    scoreSum = 0.0
    for score in scoreList:
        if not math.isnan(scoreList[score]):
            validScores += 1
            scoreSum += scoreList[score]
    if validScores == 0:
        return float('nan')
    return scoreSum / validScores


# Calculate and return IOU score for a particular category
def getIouScoreForCategory(category, confMatrix, args):
    # All labels in this category
    labels = category2labels[category]
    # The IDs of all valid labels in this category
    labelIds = [label.id for label in labels if not label.ignoreInEval and label.id in args.evalLabels]
    # If there are no valid labels, then return NaN
    if not labelIds:
        return float('nan')

    # the number of true positive pixels for this category
    # this is the sum of all entries in the confusion matrix
    # where row and column belong to a label ID of this category
    tp = np.longlong(confMatrix[labelIds, :][:, labelIds].sum())

    # the number of false negative pixels for this category
    # that is the sum of all rows of labels within this category
    # minus the number of true positive pixels
    fn = np.longlong(confMatrix[labelIds, :].sum()) - tp

    # the number of false positive pixels for this category
    # we count the column sum of all labels within this category
    # while skipping the rows of ignored labels and of labels within this category
    notIgnoredAndNotInCategory = [l for l in args.evalLabels if
                                  not id2label[l].ignoreInEval and id2label[l].category != category]
    fp = np.longlong(confMatrix[notIgnoredAndNotInCategory, :][:, labelIds].sum())

    # the denominator of the IOU score
    denom = (tp + fp + fn)
    if denom == 0:
        return float('nan')

    # return IOU
    return float(tp) / denom


# Calculate and return IOU score for a particular category
def getInstanceIouScoreForCategory(category, confMatrix, instStats, args):
    if not category in instStats["categories"]:
        return float('nan')
    labelIds = instStats["categories"][category]["labelIds"]

    tp = instStats["categories"][category]["tpWeighted"]
    fn = instStats["categories"][category]["fnWeighted"]

    # the number of false positive pixels for this category
    # same as above
    notIgnoredAndNotInCategory = [l for l in args.evalLabels if
                                  not id2label[l].ignoreInEval and id2label[l].category != category]
    fp = np.longlong(confMatrix[notIgnoredAndNotInCategory, :][:, labelIds].sum())

    # the denominator of the IOU score
    denom = (tp + fp + fn)
    if denom == 0:
        return float('nan')

    # return IOU
    return float(tp) / denom


# create a dictionary containing all relevant results
def createResultDict(confMatrix, classScores, classInstScores, categoryScores, categoryInstScores,
                     perImageStats, args):
    # write JSON result file
    wholeData = {}
    wholeData["confMatrix"] = confMatrix.tolist()
    wholeData["priors"] = {}
    wholeData["labels"] = {}
    for label in args.evalLabels:
        wholeData["priors"][id2label[label].name] = getPrior(label, confMatrix)
        wholeData["labels"][id2label[label].name] = label
    wholeData["classScores"] = classScores
    wholeData["classInstScores"] = classInstScores
    wholeData["categoryScores"] = categoryScores
    wholeData["categoryInstScores"] = categoryInstScores
    wholeData["averageScoreClasses"] = getScoreAverage(classScores, args)
    wholeData["averageScoreInstClasses"] = getScoreAverage(classInstScores, args)
    wholeData["averageScoreCategories"] = getScoreAverage(categoryScores, args)
    wholeData["averageScoreInstCategories"] = getScoreAverage(categoryInstScores, args)

    if perImageStats:
        wholeData["perImageScores"] = perImageStats

    return wholeData


def writeJSONFile(wholeData, args):
    path = os.path.dirname(args.exportFile)
    ensurePath(path)
    writeDict2JSON(wholeData, args.exportFile)


# Print confusion matrix
def printConfMatrix(confMatrix, args):
    # print line
    print("\b{text:{fill}>{width}}".format(width=15, fill='-', text=" "), end=' ')
    for label in args.evalLabels:
        print("\b{text:{fill}>{width}}".format(width=args.printRow + 2, fill='-', text=" "), end=' ')
    print("\b{text:{fill}>{width}}".format(width=args.printRow + 3, fill='-', text=" "))

    # print label names
    print("\b{text:>{width}} |".format(width=13, text=""), end=' ')
    for label in args.evalLabels:
        print("\b{text:^{width}} |".format(width=args.printRow, text=id2label[label].name[0]), end=' ')
    print("\b{text:>{width}} |".format(width=6, text="Prior"))

    # print line
    print("\b{text:{fill}>{width}}".format(width=15, fill='-', text=" "), end=' ')
    for label in args.evalLabels:
        print("\b{text:{fill}>{width}}".format(width=args.printRow + 2, fill='-', text=" "), end=' ')
    print("\b{text:{fill}>{width}}".format(width=args.printRow + 3, fill='-', text=" "))

    # print matrix
    for x in range(0, confMatrix.shape[0]):
        if (not x in args.evalLabels):
            continue
        # get prior of this label
        prior = getPrior(x, confMatrix)
        # skip if label does not exist in ground truth
        if prior < 1e-9:
            continue

        # print name
        name = id2label[x].name
        if len(name) > 13:
            name = name[:13]
        print("\b{text:>{width}} |".format(width=13, text=name), end=' ')
        # print matrix content
        for y in range(0, len(confMatrix[x])):
            if (not y in args.evalLabels):
                continue
            matrixFieldValue = getMatrixFieldValue(confMatrix, x, y, args)
            print(getColorEntry(matrixFieldValue, args) + "\b{text:>{width}.2f}  ".format(width=args.printRow,
                                                                                          text=matrixFieldValue) + args.nocol,
                  end=' ')
        # print prior
        print(getColorEntry(prior, args) + "\b{text:>{width}.4f} ".format(width=6, text=prior) + args.nocol)
    # print line
    print("\b{text:{fill}>{width}}".format(width=15, fill='-', text=" "), end=' ')
    for label in args.evalLabels:
        print("\b{text:{fill}>{width}}".format(width=args.printRow + 2, fill='-', text=" "), end=' ')
    print("\b{text:{fill}>{width}}".format(width=args.printRow + 3, fill='-', text=" "), end=' ')


# Print intersection-over-union scores for all classes.
def printClassScores(scoreList, instScoreList, args):
    if (args.quiet):
        return
    print(args.bold + "classes          IoU      nIoU" + args.nocol)
    print("--------------------------------")
    for label in args.evalLabels:
        if (id2label[label].ignoreInEval):
            continue
        labelName = str(id2label[label].name)
        iouStr = getColorEntry(scoreList[labelName], args) + "{val:>5.6f}".format(
            val=scoreList[labelName]) + args.nocol
        niouStr = getColorEntry(instScoreList[labelName], args) + "{val:>5.6f}".format(
            val=instScoreList[labelName]) + args.nocol
        print("{:<14}: ".format(labelName) + iouStr + "    " + niouStr)


# Print intersection-over-union scores for all categorys.
def printCategoryScores(scoreDict, instScoreDict, args):
    if (args.quiet):
        return
    print(args.bold + "categories       IoU      nIoU" + args.nocol)
    print("--------------------------------")
    for categoryName in scoreDict:
        if all(label.ignoreInEval for label in category2labels[categoryName]):
            continue
        iouStr = getColorEntry(scoreDict[categoryName], args) + "{val:>5.6f}".format(
            val=scoreDict[categoryName]) + args.nocol
        niouStr = getColorEntry(instScoreDict[categoryName], args) + "{val:>5.6f}".format(
            val=instScoreDict[categoryName]) + args.nocol
        print("{:<14}: ".format(categoryName) + iouStr + "    " + niouStr)

####################################################################################################
#Inf fun
####################################################################################################
class EvalPixel():
    def __init__(self, args, predictionImgList = None, groundTruthImgList = None):
        self.args = args
        self.predictionImgList = predictionImgList
        self.groundTruthImgList = groundTruthImgList
        if predictionImgList is None or groundTruthImgList is None:
            self.groundTruthImgList,  self.predictionImgList = self.getDefaultData(self.args)
            
    # evaluate image in two lists
    def evaluateImgLists(self,predictionImgList, groundTruthImgList, args):
        if len(predictionImgList) != len(groundTruthImgList):
            printError("List of images for prediction and groundtruth are not of equal size.")
        confMatrix = generateMatrix(args)
        instStats = generateInstanceStats(args)
        perImageStats = {}
        nbPixels = 0

        if not args.quiet:
            print("Evaluating {} pairs of images...".format(len(predictionImgList)))

        # Evaluate all pairs of images and save them into a matrix
        for i in range(len(predictionImgList)):
            predictionImgFileName = predictionImgList[i]
            groundTruthImgFileName = groundTruthImgList[i]
            # print "Evaluate ", predictionImgFileName, "<>", groundTruthImgFileName
            ##################################################################################################################################
            # infer mask code here
            ###################################################################################################################################
            nbPixels += self.evaluatePair(predictionImgFileName, groundTruthImgFileName, confMatrix, instStats,
                                     perImageStats, args)
            print(nbPixels)

            # sanity check

            if confMatrix.sum() != nbPixels:
                pass
                # printError(
                #     'Number of analyzed pixels and entries in confusion matrix disagree: contMatrix {}, pixels {}'.format(
                #         confMatrix.sum(), nbPixels))

            if not args.quiet:
                print("\rImages Processed: {}".format(i + 1), end=' ')
                sys.stdout.flush()
                
            #########################################################################################debug True##############
            print("finish debug")
            break
        if not args.quiet:
            print("\n")

        # sanity check
        if confMatrix.sum() != nbPixels:
            pass
        #     printError(
        #         'Number of analyzed pixels and entries in confusion matrix disagree: contMatrix {}, pixels {}'.format(
        #             confMatrix.sum(), nbPixels))

        # print confusion matrix
        if (not args.quiet):
            printConfMatrix(confMatrix, args)

        # print accuracy
        # Calculate and return IOU score for a particular label
        # pdb.set_trace()
        
        acc = np.diag(confMatrix).sum() / confMatrix.sum()
        print("pixel accuracy")
        print(acc)

        # Calculate IOU scores on class level from matrix
        classScoreList = {}
        for label in args.evalLabels:
            labelName = id2label[label].name
            classScoreList[labelName] = getIouScoreForLabel(label, confMatrix, args)

        # Calculate instance IOU scores on class level from matrix
        classInstScoreList = {}
        for label in args.evalLabels:
            labelName = id2label[label].name
            classInstScoreList[labelName] = getInstanceIouScoreForLabel(label, confMatrix, instStats, args)

        # Print IOU scores
        if (not args.quiet):
            print("")
            print("")
            printClassScores(classScoreList, classInstScoreList, args)
            iouAvgStr = getColorEntry(getScoreAverage(classScoreList, args), args) + "{avg:5.6f}".format(
                avg=getScoreAverage(classScoreList, args)) + args.nocol
            niouAvgStr = getColorEntry(getScoreAverage(classInstScoreList, args), args) + "{avg:5.6f}".format(
                avg=getScoreAverage(classInstScoreList, args)) + args.nocol
            print("--------------------------------")
            print("Score Average : " + iouAvgStr + "    " + niouAvgStr)
            print("--------------------------------")
            print("")

        # Calculate IOU scores on category level from matrix
        categoryScoreList = {}
        for category in category2labels.keys():
            categoryScoreList[category] = getIouScoreForCategory(category, confMatrix, args)

        # Calculate instance IOU scores on category level from matrix
        categoryInstScoreList = {}
        for category in category2labels.keys():
            categoryInstScoreList[category] = getInstanceIouScoreForCategory(category, confMatrix, instStats, args)

        # Print IOU scores
        if (not args.quiet):
            print("")
            printCategoryScores(categoryScoreList, categoryInstScoreList, args)
            iouAvgStr = getColorEntry(getScoreAverage(categoryScoreList, args), args) + "{avg:5.6f}".format(
                avg=getScoreAverage(categoryScoreList, args)) + args.nocol
            niouAvgStr = getColorEntry(getScoreAverage(categoryInstScoreList, args), args) + "{avg:5.6f}".format(
                avg=getScoreAverage(categoryInstScoreList, args)) + args.nocol
            print("--------------------------------")
            print("Score Average : " + iouAvgStr + "    " + niouAvgStr)
            print("--------------------------------")
            print("")

        # write result file
        allResultsDict = createResultDict(confMatrix, classScoreList, classInstScoreList, categoryScoreList,
                                          categoryInstScoreList, perImageStats, args)
        # writeJSONFile(allResultsDict, args)

        # return confusion matrix
        return allResultsDict

    # Main evaluation method. Evaluates pairs of prediction and ground truth
    # images which are passed as arguments.
    def evaluatePair(self,predictionImgFileName, groundTruthImgFileName, confMatrix, instanceStats, perImageStats, args):
        # Loading all resources for evaluation.
        try:
            predictionImg = Image.open(predictionImgFileName)
            predictionNp = np.array(predictionImg)
        except:
            printError("Unable to load " + predictionImgFileName)
        try:
            groundTruthImg = Image.open(groundTruthImgFileName)
            groundTruthNp = np.array(groundTruthImg)
        except:
            printError("Unable to load " + groundTruthImgFileName)
        # load ground truth instances, if needed
        if args.evalInstLevelScore:
            groundTruthInstanceImgFileName = groundTruthImgFileName.replace("labelIds", "instanceIds")
            try:
                instanceImg = Image.open(groundTruthInstanceImgFileName)
                instanceNp = np.array(instanceImg)
            except:
                printError("Unable to load " + groundTruthInstanceImgFileName)

        # Check for equal image sizes
        if (predictionImg.size[0] != groundTruthImg.size[0]):
            printError(
                "Image widths of " + predictionImgFileName + " and " + groundTruthImgFileName + " are not equal.")
        if (predictionImg.size[1] != groundTruthImg.size[1]):
            printError(
                "Image heights of " + predictionImgFileName + " and " + groundTruthImgFileName + " are not equal.")
        if (len(predictionNp.shape) != 2):
            printError("Predicted image has multiple channels.")

        imgWidth = predictionImg.size[0]
        imgHeight = predictionImg.size[1]
        nbPixels = imgWidth * imgHeight

        # Evaluate images
        if (CSUPPORT):
            # using cython
            confMatrix = addToConfusionMatrix.cEvaluatePair(predictionNp, groundTruthNp, confMatrix, args.evalLabels)

        else:
            # the slower python way
            for (groundTruthImgPixel, predictionImgPixel) in izip(groundTruthImg.getdata(), predictionImg.getdata()):
                if (not groundTruthImgPixel in args.evalLabels):
                    printError("Unknown label with id {:}".format(groundTruthImgPixel))

                confMatrix[groundTruthImgPixel][predictionImgPixel] += 1

        if args.evalInstLevelScore:
            # Generate category masks
            categoryMasks = {}
            for category in instanceStats["categories"]:
                categoryMasks[category] = np.in1d(predictionNp,
                                                  instanceStats["categories"][category]["labelIds"]).reshape(
                    predictionNp.shape)

            instList = np.unique(instanceNp[instanceNp > 1000])
            for instId in instList:
                labelId = int(instId / 1000)
                label = id2label[labelId]
                if label.ignoreInEval:
                    continue

                mask = instanceNp == instId
                instSize = np.count_nonzero(mask)

                tp = np.count_nonzero(predictionNp[mask] == labelId)
                fn = instSize - tp

                weight = args.avgClassSize[label.name] / float(instSize)
                tpWeighted = float(tp) * weight
                fnWeighted = float(fn) * weight

                instanceStats["classes"][label.name]["tp"] += tp
                instanceStats["classes"][label.name]["fn"] += fn
                instanceStats["classes"][label.name]["tpWeighted"] += tpWeighted
                instanceStats["classes"][label.name]["fnWeighted"] += fnWeighted

                category = label.category
                if category in instanceStats["categories"]:
                    catTp = 0
                    catTp = np.count_nonzero(np.logical_and(mask, categoryMasks[category]))
                    catFn = instSize - catTp

                    catTpWeighted = float(catTp) * weight
                    catFnWeighted = float(catFn) * weight

                    instanceStats["categories"][category]["tp"] += catTp
                    instanceStats["categories"][category]["fn"] += catFn
                    instanceStats["categories"][category]["tpWeighted"] += catTpWeighted
                    instanceStats["categories"][category]["fnWeighted"] += catFnWeighted

        if True: # evaluate pixel accuracy
            notIgnoredLabels = [l for l in args.evalLabels if not id2label[l].ignoreInEval]
            notIgnoredPixels = np.in1d(groundTruthNp, notIgnoredLabels, invert=True).reshape(groundTruthNp.shape)
            erroneousPixels = np.logical_and(notIgnoredPixels, (predictionNp != groundTruthNp))
            perImageStats[predictionImgFileName] = {}
            perImageStats[predictionImgFileName]["nbNotIgnoredPixels"] = np.count_nonzero(notIgnoredPixels)
            perImageStats[predictionImgFileName]["nbCorrectPixels"] = np.count_nonzero(erroneousPixels)
            # print("all pixel_count:"+str(perImageStats[predictionImgFileName]["nbNotIgnoredPixels"]))
            # print("correct pixel_count:"+str(perImageStats[predictionImgFileName]["nbCorrectPixels"]))
            # print("pixel_accuracy:"+str((float(perImageStats[predictionImgFileName]["nbCorrectPixels"])/perImageStats[predictionImgFileName]["nbNotIgnoredPixels"])))

        return nbPixels


    # launch the process
    def run(self):
        self.evaluateImgLists(self.predictionImgList, self.groundTruthImgList, self.args)

    # get the default data
    def getDefaultData(self, args):
        groundTruthImgList, predictionImgList = [], []
        groundTruthImgList = glob.glob(args.groundTruthSearch)
        if not groundTruthImgList:
            printError("Cannot find any ground truth images to use for evaluation. Searched for: {}".format(
                args.groundTruthSearch))
        # get the corresponding prediction for each ground truth imag
        for gt in groundTruthImgList:
            predictionImgList.append(getPrediction(args, gt))
        return groundTruthImgList, predictionImgList


class CityscapesEvaluator(object):

    def evaluate(self, pred_dir=None, gt_dir=None):
        """
        :param pred_dir: directory of model output results(must be consistent with val directory)
        :param gt_dir: directory of  cityscapes data(root)
        :return:
        """
        pred_path = pred_dir
        data_path = gt_dir
        print("evaluate the result...")
        args = CArgs(data_path=data_path, out_path=data_path, predict_path=pred_path)
        ob = EvalPixel(args)
        ob.run()


In [16]:
from lib.utils.helpers.file_helper import FileHelper
from lib.utils.helpers.image_helper import ImageHelper
from lib.utils.tools.logger import Logger as Log
from lib.metrics.running_score import RunningScore
from lib.vis.seg_visualizer import SegVisualizer
from lib.vis.palette import get_cityscapes_colors
from segmentor.tools.module_runner import ModuleRunner
from segmentor.tools.optim_scheduler import OptimScheduler
from pathlib import Path
import torch.nn.functional as F
import timeit
import pdb
import cv2

import fnmatch
import platform


if __name__ == "__main__":
    INFERENCE = True
    parser = argparse.ArgumentParser()
    if CFG.model_method == 'fcn':
        if CFG.phase == 'infer':
            print('Load Model Now ...')
            configer = Configer(args_parser = main())
            data_dir = configer.get('data', 'data_dir')
            
            if isinstance(data_dir, str):
                data_dir = [data_dir]
            abs_data_dir = [os.path.expanduser(x) for x in data_dir]
            configer.update(['data', 'data_dir'], abs_data_dir)
            
            #python
            #project_dir = os.path.dirname(os.path.realpath(__file__))
            #jupyter
            project_dir = os.path.dirname(Path().resolve())
            configer.add(['project_dir'], project_dir)
            model = None
            
            model =Tester(configer)
            model.test()
            print('Load Model Finish')
            
            if INFERENCE:
                print('Inference Starting ...')
                args = inference_fn()
                cityscapes_evaluator = CityscapesEvaluator()
                cityscapes_evaluator.evaluate(pred_dir=args.pred_dir, gt_dir=args.gt_dir)
                
                print(prototype)
  
        else:
            print('train module will be available')
    else:
        print('attention models will be available')

2022-11-05 15:43:43,451 INFO    [module_runner.py, 46] BN Type is torchsyncbn.


Load Model Now ...
hrnet_w48_proto
ok


2022-11-05 15:43:44,793 INFO    [module_runner.py, 87] Loading checkpoint from /tmp/working/workspace/ProtoSeg_local/output/Cityscapes/checkpoints/cityscapes/hrnet_w48_proto_lr1x_hrnet_proto_80k_latest.pth...
2022-11-05 15:43:44,945 INFO    [module_runner.py, 117] {'dataset': 'cityscapes', 'method': 'fcn_segmentor', 'data': {'image_tool': 'cv2', 'input_mode': 'BGR', 'num_classes': 19, 'label_list': [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33], 'data_dir': ['/tmp/working/workspace/ProtoSeg_local/data/Cityscapes'], 'workers': 8, 'include_val': False, 'include_coarse': False, 'only_coarse': False, 'only_mapillary': False, 'only_small': False, 'include_atr': False, 'include_cihp': False, 'drop_last': True}, 'train': {'batch_size': 8, 'data_transformer': {'size_mode': 'fix_size', 'input_size': [1024, 512], 'align_method': 'only_pad', 'pad_mode': 'random'}}, 'val': {'batch_size': 4, 'mode': 'ss_test', 'data_transformer': {'size_mode': 'fix_size', 'input_size': [

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-3efdd166a418>", line 41, in <module>
    model.test()
  File "<ipython-input-13-bb083826d099>", line 76, in test
    for j, data_dict in enumerate(self.test_loader):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 517, in __next__
    data = self._next_data()
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1199, in _next_data
    return self._process_data(data)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1225, in _process_data
    data.reraise()
  File "/opt/conda/lib/python3.8/site-packages/torch/_utils.py", line 429, in reraise
    raise self.exc_type(msg)
RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent ca

TypeError: object of type 'NoneType' has no len()